# VIE Airport Türen

In [1]:
from pandas import read_excel, concat, to_numeric
from datetime import datetime as dt

levels = {
    '0':0,
    '- 1':-1,
    '1':1,
    '2':2,
    '3':3,
    '4':4,
    '- 2':-2,
    'ZG':0.5,
    'K':-1,
    '5':5,
    '6':6,
    '8':8,
    '7':7,
    'DG':99,
    'ZG1':1.5,
    'EG':0,
    '9':9,
    '10':10,
    'ZG2':2.5,
    '21':21,
    '19':19,
    '20':20,
    '18':18,
    '17':17,
    '13':13,
    '15':15,
    '22':22,
    '11':11,
    '16':16,
    '14':14,
    '12':12,
    '-2':-2,
    '-1':-1
}

def clean_df(df):
    d = df.dropna(axis=1, how="all")
    new_columns = []
    for col in d.columns:
        col = col.lower().replace(" ", "_").replace(".", " ").replace("ü", "ue")
        col = col.replace("ä", "ae").replace("ö", "oe").replace("ß", "ss")
        col = col.replace(":", "").replace("?", "")
        new_columns.append(col)
    d.columns = new_columns
    return d

int_mapper = lambda x: int(x)
x_mapper = lambda x: True if x=="X" else False
object_floor_mapper = lambda x: x.split("_")[0]
level_mapper = lambda x: levels[x] if x in levels.keys() else 99
date_mapper = lambda x: dt.strptime(x, "%d.%m.%y")

## Haltemagnete Schrack

In [2]:
file_name = "Schrack_HM_Obj420.xls"
columns = [
    "object",
    "level",
    "door_id",
    "door_id_alt",
    "bst_door_id_alt",
    "room_id",
    "aks_room_id",
    "uz6_control",
    "door_series",
    "lfd_id",
    "info"
]

hm_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="HALTEMAGNETE"
)

hm_data.columns = columns

# Add origin
hm_data["origin"] = file_name

# Data cleaning
hm_data.dropna(inplace=True)
hm_data["object"] = hm_data["object"].map(int_mapper)
hm_data["uz6_control"] = hm_data["uz6_control"].map(int_mapper)
hm_data["door_series"] = hm_data["door_series"].map(int_mapper)
hm_data["lfd_id"] = hm_data["lfd_id"].map(int_mapper)

hm_data.head(5)

,object,level,door_id,door_id_alt,bst_door_id_alt,room_id,aks_room_id,uz6_control,door_series,lfd_id,info,origin
7,420,EG,420 EG0A2207.01,017,420/0/40/6,017,0A2207,7,3,1,Spannungs-\nunterbrechung\nvon 17.00-06.30 übe...,Schrack_HM_Obj420.xls
9,420,EG,420 EG0A1206.01,018,420/0/45/6,008,0A1206,8,4,1,Spannungs-\nunterbrechung\nvon 17.00-06.30 übe...,Schrack_HM_Obj420.xls
11,420,EG,420 EG0A1308.01,011A,420/0/29/6,011A,0A1308,9,5,1,HM demontiert /\nVerbindungsdose,Schrack_HM_Obj420.xls
12,420,EG,420 EG0A1308.02,037A,420/0/30/6,013,0A1607,9,5,2,HM demontiert /\nVerbindungsdose,Schrack_HM_Obj420.xls
65,420,EG,keine vorhanden,keine vorhanden,Keine,0103,0B0304,121,29,1,Freilager EX,Schrack_HM_Obj420.xls


## Brandschutztechnik (BST) Sisando

In [3]:
file_name = "Sisando_BST_Obj420.xlsx"

columns = [
    "object",
    "level",
    "door_id",
    "status_text",
    "status",
    "door_class",
    "room_id",
    "room_id_additional",
    "door_audit_remark",
    "door_audit_date",
    "door_audit_responsible_short",
    "door_audit_responsible_long",
    "electrical_operation",
    "exf",
    "retention_mechanism",
    "manufacturer",
    "classification"
]

bst_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="Sheet1",
    parse_dates=[9]
)

bst_data.columns = columns

# Data Cleaning
bst_data["object"] = bst_data["object"].map(int_mapper)

# Add origin
bst_data["origin"] = file_name

bst_data.head(5)

,object,level,door_id,status_text,status,door_class,room_id,room_id_additional,door_audit_remark,door_audit_date,door_audit_responsible_short,door_audit_responsible_long,electrical_operation,exf,retention_mechanism,manufacturer,classification,origin
0,420,0,A0201.01,OK,1,Drehflügel #1,A0201,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,NaN,NaN,Peneder,T30,Sisando_BST_Obj420.xlsx
1,420,0,A0203.01,OK,1,Drehflügel #2,A0203,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Peneder,EI2-30-C,Sisando_BST_Obj420.xlsx
2,420,0,A0803.01,OK,1,Drehflügel #1,A0803,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Peneder,EI2-30-C,Sisando_BST_Obj420.xlsx
3,420,0,A0803.02,HINWEIS,21,Drehflügel #2,A0803,NaN,STATUS OK_x001D_H000 Freilauftürschließer hält...,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Fritscher,T30,Sisando_BST_Obj420.xlsx
4,420,0,A0807.03,OK,1,Drehflügel #2,A0807,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,NaN,NaN,Fritscher,T30,Sisando_BST_Obj420.xlsx


## Feuerleittechnik (FLT) Sisando

In [4]:
file_name = "Sisando_FLT_Obj420.xlsx"

columns = [
    "row_id",
    "object",
    "module",
    "section",
    "floor",
    "room_id",
    "number",
    "is_emergency_door",
    "is_emergency_door_no",
    "has_blind_cylinder",
    "has_cylinder",
    "has_panic_lock",
    "is_locked",
    "is_lockable",
    "is_not_lockable",
    "has_emergency_button",
    "has_emergency_button_no",
    "defect_memo",
    "defect_fixed",
    "none_a",
    "none_b",
    "none_c"
]

flt_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="all",
    skiprows=[0,1],
    header=None
)

flt_data.columns = columns

flt_data.drop("is_emergency_door_no", inplace=True, axis=1)
flt_data.drop("row_id", inplace=True, axis=1)
flt_data.drop("has_emergency_button_no", axis=1, inplace=True)
flt_data.drop("is_not_lockable", axis=1, inplace=True)
flt_data.drop("none_a", axis=1, inplace=True)
flt_data.drop("none_b", axis=1, inplace=True)
flt_data.drop("none_c", axis=1, inplace=True)

# Deleting of unused rows
flt_data["has_emergency_button"] = flt_data["has_emergency_button"].map(x_mapper)
flt_data["is_emergency_door"] = flt_data["is_emergency_door"].map(x_mapper)
flt_data["has_blind_cylinder"] = flt_data["has_blind_cylinder"].map(x_mapper)
flt_data["has_cylinder"] = flt_data["has_cylinder"].map(x_mapper)
flt_data["has_panic_lock"] = flt_data["has_panic_lock"].map(x_mapper)
flt_data["is_locked"] = flt_data["is_locked"].map(x_mapper)
flt_data["is_lockable"] = flt_data["is_lockable"].map(x_mapper)
flt_data["has_panic_lock"] = flt_data["has_panic_lock"].map(x_mapper)
flt_data["defect_fixed"] = flt_data["defect_fixed"].map(x_mapper)
flt_data["object"] = flt_data["object"].map(object_floor_mapper)

# Add origin
flt_data["origin"] = file_name

flt_data.head(5)

,object,module,section,floor,room_id,number,is_emergency_door,has_blind_cylinder,has_cylinder,has_panic_lock,is_locked,is_lockable,has_emergency_button,defect_memo,defect_fixed,origin
0,420,A,EG,A,3108,01,True,False,True,False,False,True,False,NaN,False,Sisando_FLT_Obj420.xlsx
1,420,A,EG,NaN,5206,05,False,False,True,False,False,True,False,"Panikschloss fehlt (Außentür, Offline-Beschlag?)",False,Sisando_FLT_Obj420.xlsx
2,420,A,EG,NaN,4401,07,True,False,False,False,False,False,False,NaN,False,Sisando_FLT_Obj420.xlsx
3,420,A,EG,NaN,2601,06,True,False,True,False,False,True,False,NaN,False,Sisando_FLT_Obj420.xlsx
4,420,A,EG,NaN,2601,04,True,False,True,False,False,True,False,NaN,False,Sisando_FLT_Obj420.xlsx


## Filemaker Database

In [5]:
file_name = "data/BSTüren_Filemaker_11.04.22.xlsx"
filemaker_data = clean_df(read_excel(file_name, sheet_name="Gesamt"))

filemaker_data["ebene"] = filemaker_data["ebene"].map(level_mapper)

flt_data["origin"] = file_name

filemaker_data.head()

,bauteil,ebene,topnr,brandmelderkette,brandmeldernr,erfasst_von,tuer_12flg,zugaengigkeit,ok,hersteller,...,suanbpos,nettacount,tax,gesuanbot,arbeitszeit,material,schadensmeldung,aufnahme,kleinregie_summe,zyklus
0,063,-1.0,1.0,UZ11,NaN,Traxler,NaN,NaN,NaN,PENEDER,...,220.0,5651677.5,1130335.5,6782013,NaN,NaN,NaN,744,10,NaN
1,063,-1.0,2.0,UZ11,NaN,Traxler,NaN,NaN,NaN,PENEDER,...,220.0,5651677.5,1130335.5,6782013,NaN,NaN,NaN,744,10,NaN
2,063,-1.0,3.0,NaN,D03,te Best,NaN,NaN,NaN,UNBEKANNT,...,16080.0,5651677.5,1130335.5,6782013,NaN,NaN,NaN,744,10,NaN
3,063,-1.0,4.0,UZ11,NaN,Traxler,NaN,NaN,NaN,UNBEKANNT,...,NaN,5651677.5,1130335.5,6782013,NaN,NaN,NaN,744,10,NaN
4,063,-1.0,5.0,UZ11,NaN,NaN,NaN,NaN,NaN,UNBEKANNT,...,220.0,5651677.5,1130335.5,6782013,NaN,NaN,NaN,744,10,NaN


## Integration of data